#Spark Apache (семинары)

###Урок 3. Инструменты работы и визуализации ч.2

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

("2023-11-20", "Electronics", 100, 12000),
("2023-11-21", "Electronics", 110, 13000),
("2023-11-22", "Electronics", 105, 12500),
("2023-11-20", "Clothing", 300, 15000),
("2023-11-21", "Clothing", 280, 14000),
("2023-11-22", "Clothing", 320, 16000),
("2023-11-20", "Books", 150, 9000),
("2023-11-21", "Books", 200, 12000),
("2023-11-22", "Books", 180, 10000)

Столбцы: "date", "category", "quantity", "revenue".

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.
Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [ ]:
!pip install pyspark

In [21]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, mean, round
from pyspark.sql.window import Window

In [3]:
data = ("2023-11-20", "Electronics", 100, 12000), ("2023-11-21", "Electronics", 110, 13000), ("2023-11-22", "Electronics", 105, 12500), ("2023-11-20", "Clothing", 300, 15000), ("2023-11-21", "Clothing", 280, 14000), ("2023-11-22", "Clothing", 320, 16000), ("2023-11-20", "Books", 150, 9000), ("2023-11-21", "Books", 200, 12000), ("2023-11-22", "Books", 180, 10000)
df = pd.DataFrame(data, columns=["date", "category", "quantity", "revenue"])

In [36]:
spark = SparkSession.builder.appName('dz2').getOrCreate()
dfs = spark.createDataFrame(df)
# С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта
dfs.withColumn('mean_revenue', mean(col("revenue")*col('quantity')).over(Window.partitionBy('category'))).show()

#Примените операцию pivot
dfs.groupBy('category').pivot('date').agg(mean(col("revenue")*col('quantity'))).show()

+----------+-----------+--------+-------+------------------+
|      date|   category|quantity|revenue|      mean_revenue|
+----------+-----------+--------+-------+------------------+
|2023-11-20|      Books|     150|   9000|         1850000.0|
|2023-11-21|      Books|     200|  12000|         1850000.0|
|2023-11-22|      Books|     180|  10000|         1850000.0|
|2023-11-20|   Clothing|     300|  15000| 4513333.333333333|
|2023-11-21|   Clothing|     280|  14000| 4513333.333333333|
|2023-11-22|   Clothing|     320|  16000| 4513333.333333333|
|2023-11-20|Electronics|     100|  12000|1314166.6666666667|
|2023-11-21|Electronics|     110|  13000|1314166.6666666667|
|2023-11-22|Electronics|     105|  12500|1314166.6666666667|
+----------+-----------+--------+-------+------------------+

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|Electronics| 1200000.0| 1430000.0| 1312500.0|
|   Clothing| 4500

In [35]:
spark.stop()